In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mimg
from glob import glob

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh
import cv2

In [ ]:
def get_fn(face):
    return glob(f"CFD Version 3.0/Images/CFD/{face}/*")[0]
    
def get_img(face):
    return cv2.imread(get_fn(face))[:, :, ::-1]
    #return  mimg.imread(get_fn(face))

def plot_img(ax, img):
    ax.imshow(img)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return ax

def plot(ax, df, x, y, i, group='GenderSelf', vlines=[0.36, 1.5, 5.5, 'r']):
    if not group:
        ax.plot(x, y, '.')
    else:
        group_dict = df.groupby(group).groups
        for label, idx in group_dict.items():
            ax.plot(x[idx], y[idx], '.', label=label)
    ax.plot(x[i], y[i], 'k*', ms=10)
    ax.vlines(*vlines)
    ax.legend()
    ax.set_ylabel('Attractive')
    ax.set_xlabel('length ratio')
    ax.grid()
    return ax

# def to_array(p):
#     return np.array([p.x, p.y, p.z])

In [ ]:
def draw(image, results):
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
        #print('face_landmarks:', face_landmarks)
        mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
        mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
        mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
    return annotated_image

# def mean(points):
#     return np.mean(np.array([to_array(p) for p in points]), axis=0)

# def get_points(results):
#     landmarks = results.multi_face_landmarks[0]
#     p_lefteye = mean([landmarks.landmark[i] for i in left_eye])
#     p_righteye = mean([landmarks.landmark[i] for i in right_eye])
#     p_left = to_array(landmarks.landmark[356])
#     p_right = to_array(landmarks.landmark[127])
#     return p_lefteye, p_righteye, p_left, p_right

# def compute_w_ratio(results):
#     p_lefteye, p_righteye, p_left, p_right = get_points(results)
#     return np.linalg.norm(p_lefteye - p_righteye) / np.linalg.norm(p_left - p_right)
from utils import compute_w_ratio, get_points, mean

def classify_lr(x, y):
    if y < x.quantile(1/3):
        return "low"
    elif y > x.quantile(2/3):
        return "high"
    else:
        return "medium"

In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

left_eye = [i[1] for i in mp.solutions.face_mesh.FACEMESH_LEFT_EYE]
right_eye = [i[0] for i in mp.solutions.face_mesh.FACEMESH_RIGHT_EYE]

# def get_width_ratio(results):
#     landmarks = results.multi_face_landmarks[0]
#     x_left, y_left = mean([landmarks.landmark[i] for i in left_eye])
#     x_right, y_right = mean([landmarks.landmark[i] for i in right_eye])
#     p_left = landmarks.landmark[356]
#     p_right = landmarks.landmark[127]
#     return (x_left - x_right) / (p_left.x - p_right.x)

In [ ]:
fn_excel = "CFD Version 3.0/CFD 3.0 Norming Data and Codebook.xlsx"

df = pd.read_excel(fn_excel, sheet_name='CFD U.S. Norming Data', header=7).loc[1:]
df

# summary = df.describe()
# summary.loc[:, ~np.isnan(summary.loc['mean'])]

In [ ]:
y = df['Attractive']
x = df['UpperFaceLength2'] / df['FaceLength']
df['lr'] = x

In [ ]:
df.EthnicitySelf.unique()

In [ ]:
x.quantile(q=1/3), x.quantile(q=2/3), x.quantile()
y.mean()

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(20, 7)

i = np.random.randint(0, len(df))
model = df.loc[i].Model
img = get_img(model)
plot(ax[0], df, x, y, i, group='EthnicitySelf')

# ax[0].vlines(x.quantile(1/3), 1.5, 5.5, 'k')
# ax[0].vlines(x.quantile(2/3), 1.5, 5.5, 'k')
# ax[0].hlines(y.quantile(.25), 0.3, 0.45, 'k')
# ax[0].hlines(y.quantile(.75), 0.3, 0.45, 'k')

plot_img(ax[1], img)

In [ ]:
# For static images:
IMAGE_FILES = []
IMAGE_FILES = [get_fn(model)]
# IMAGE_FILES = df.Model.apply(get_fn)

meshs = {}
IMAGE_FILES = df.Model
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
      for idx, file in enumerate(IMAGE_FILES):
            print(file, end='.')
            fn = get_fn(file)
            image = cv2.imread(fn)
            # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            meshs[file] = results

            # Print and draw face mesh landmarks on the image.
            if not results.multi_face_landmarks:
                  continue

            #annotated_image = draw(image, results)
            #cv2.imwrite(f'annotated_image_{model}.png', annotated_image)
            print(compute_w_ratio(results))
            #print(get_width_ratio(results))

In [ ]:
i = np.random.randint(0, len(df))
model = df.loc[i].Model
results = meshs[model]
compute_w_ratio(results)
image = cv2.imread(get_fn(model))
image = draw(image, results)

w_ratio = compute_w_ratio(results)
p_lefteye, p_righteye, p_left, p_right = get_points(results)

height, width, channels = image.shape
plt.title(w_ratio)
plt.imshow(image[:, :, ::-1])
plt.plot(p_lefteye[0]*width, p_lefteye[1]*height, 'x')
plt.plot(p_righteye[0]*width, p_righteye[1]*height, 'x')
plt.plot(p_left[0]*width, p_left[1]*height, 'x')
plt.plot(p_right[0]*width, p_right[1]*height, 'x')

In [ ]:
# [meshs]
wrs = [compute_w_ratio(results) for model, results in meshs.items()]
df['wr'] = wrs

In [ ]:
df.wr.describe()

In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(20, 7)

i = np.random.randint(0, len(df))
model = df.loc[i].Model
img = get_img(model)
plot(ax[0], df, df.wr, y, i, group='GenderSelf', vlines=[0.46, 1.5, 5.5, 'r'])

ax[1] = plot_img(ax[1], img)
ax[0].set_xlabel('width ratio')

In [ ]:
def create_classify_function(a, b):
    def classify(v):
        if v < a:
            return "low"
        elif v < b:
            return "medium"
        else:
            return "high"
    return classify

In [ ]:
df_male = df[df.GenderSelf == 'M']
a, b = df_male.wr.quantile(1/3),  df_male.wr.quantile(2/3)
classifier = create_classify_function(a, b)
df_male['class'] = df_male.wr.apply(classifier)

In [ ]:
'class' in list(filter(lambda s: 'Unnamed' not in s, df.columns))

In [ ]:
# list(filter(lambda s: 'Unnamed' not in s, df.columns))
df_male = df_male[list(filter(lambda s: 'Unnamed' not in s, df_male.columns))]

ethn = [a + 'Prob' for a in ['Asian', 'Black', 'Latino', 'Multi', 'Other', 'White']]

# df_male.index[i], i
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    df_male_mean = df_male[ethn].mean()
    print(df_male_mean)
    
df_male_mean.sum()
df_male_mean

In [ ]:
from collections import Counter
len(df_male), 
{k: v/len(df_male) for k, v in Counter(df_male.EthnicitySelf).items()}

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_male.columns)

In [ ]:
wr_group = df_male.groupby('class').groups
# wr_grouped.groups['low']

fig, ax = plt.subplots(1, 2)
fig.set_size_inches(20, 7)

i = np.random.randint(0, len(df_male))
model = df.iloc[i].Model
img = get_img(model)
plot(ax[0], df_male, df_male.wr, y, df_male.index[i], group='class', vlines=[0.46, 1.5, 5.5, 'r'])

ax[1] = plot_img(ax[1], img)
ax[0].set_xlabel('width ratio')

In [ ]:
def compute_face_average(faces, face_n=None, max_n=300):
    print(len(faces))
    if face_n:
        faces = faces.iloc[np.random.choice(range(len(faces)), size=face_n, replace=False)]

    for i, face in enumerate(faces):
        if i == 0:
            img = get_img(face) / 256
        else:            
            img += get_img(face) / 256
        if i >= max_n:
            break
    return img / (i+1)

# def show_face_average(faces, face_n=None, max_n=300):
#     img = compute_face_average(faces, face_n=None, max_n=300)
#     plt.imshow(img/(i+1))

def to255(image):
    return np.rint(image * 256).astype(np.uint8)

In [ ]:
wr_low = df_male.loc[wr_group["low"]]
avgimg_wr_low = compute_face_average(wr_low.Model)

In [ ]:
plt.title(f'wr low: {wr_low.wr.mean()}')
plt.imshow(avgimg_wr_low)

cv2.imwrite('base_faces/avgimg_wr_low.png', to255(avgimg_wr_low)[:, :, ::-1])

In [ ]:
wr_medium = df_male.loc[wr_group["medium"]]
avgimg_wr_medium = compute_face_average(wr_medium.Model)

In [ ]:
plt.title(f'wr medium: {wr_medium.wr.mean()}')
plt.imshow(avgimg_wr_medium)
cv2.imwrite('base_faces/avgimg_wr_medium.png', to255(avgimg_wr_medium)[:, :, ::-1])

In [ ]:
wr_high = df_male.loc[wr_group["high"]]
avgimg_wr_high = compute_face_average(wr_high.Model)

In [ ]:
plt.title(f'wr medium: {wr_high.wr.mean()}')
plt.imshow(avgimg_wr_high)
cv2.imwrite('base_faces/avgimg_wr_high.png', to255(avgimg_wr_high)[:, :, ::-1])

In [ ]:
with mp_face_mesh.FaceMesh(
                                static_image_mode=True,
                                max_num_faces=1,
                                refine_landmarks=True,
                                min_detection_confidence=0.5) as face_mesh:
    print(file, end='.')
    meshs_avg = {label: face_mesh.process(cv2.cvtColor(np.rint(image * 256).astype(np.uint8), cv2.COLOR_BGR2RGB))
                 for label, image in zip(['low', 'medium', 'high'], [avgimg_wr_low, avgimg_wr_medium, avgimg_wr_high])}

In [ ]:
plt.imshow(draw(avgimg_wr_low, meshs_avg['low']))
compute_w_ratio(meshs_avg['low'])

In [ ]:
plt.imshow(draw(avgimg_wr_medium, meshs_avg['medium']))
compute_w_ratio(meshs_avg['medium'])

In [ ]:
plt.imshow(draw(avgimg_wr_high, meshs_avg['high']))
compute_w_ratio(meshs_avg['high'])